In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [2]:
url = 'https://www.skyscrapercity.com/forums/forum-pencakar-langit-malaysia.164/'

In [3]:
r = requests.get(url)
soup = BeautifulSoup(r.content, "lxml")

In [10]:
forums = [
    'https://www.skyscrapercity.com/forums/countdown-to-2030.332/',
    'https://www.skyscrapercity.com/forums/skyscrapers-buildings.363/',
    'https://www.skyscrapercity.com/forums/mixed-developments.1664/',
    'https://www.skyscrapercity.com/forums/cityscapes.1493/',
    'https://www.skyscrapercity.com/forums/pangsapuri.606/',
    'https://www.skyscrapercity.com/forums/urban.364/',
    'https://www.skyscrapercity.com/forums/architecture.669/',
    'https://www.skyscrapercity.com/forums/low-rise.925/',
    'https://www.skyscrapercity.com/forums/pengangkutan.926/'
]

In [ ]:
https://www.skyscrapercity.com/forums/countdown-to-2030.332/page-11

In [23]:
forum.split('com/')[1]

'forums/countdown-to-2030.332/'

In [28]:
forum = forums[0].split('com/')[1]
r = requests.get(forums[0])
soup = BeautifulSoup(r.content, "lxml")
a = soup.find_all('a')
a = [a_.get('href') for a_ in a if a_.get('href')]
a = [a_ for a_ in a if forum in a_ and '/page-' in a_]
max_page = []
for a_ in a:
    try:
        max_page.append(int(a_.split('/page-')[-1]))
    except:
        pass

if len(max_page):
    max_page = max(max_page)
else:
    max_page = -1

In [29]:
max_page

11

In [33]:
a = soup.find_all('a')
a = [a_.get('href') for a_ in a if a_.get('href')]
a = [a_ for a_ in a if '/threads/' in a_]
a = list(set(['/'.join(a_.split('/')[:3]) for a_ in a]))
a

['/threads/bandar-cassia-batu-kawan-penang-u-c.1854698',
 '/threads/tun-razak-exchange-kuala-lumpur-jln-tun-razak.1135763',
 '/threads/kuching-isthmus-development-general-news-updates.973986',
 '/threads/vanke-kl-serani-row-redevelopment-kuala-lumpur-jalan-raja-chulan-pro.2073073',
 '/threads/seri-tanjung-pinang-gurney-bay-penang.552100',
 '/threads/ncer-northern-corridor-economic-region-penang-kedah-perlis-sime-darby.495680',
 '/threads/armani-hartajaya-bb-kuala-lumpur-jalan-raja-chulan-prep.2071347',
 '/threads/penang-international-commercial-city-penang-bayan-baru.1973689',
 '/threads/klang-valley-shopping-malls-proposals-news-updates.1498603',
 '/threads/merdeka-118-kuala-lumpur-changkat-stadium-city-centre.956606',
 '/threads/tropicana-windcity-genting-highlands.2230624',
 '/threads/kwasa-damansara-city-centre-selangor-sg-buloh.1522715',
 '/threads/sarawak-corridor-of-renewable-energy-score.581188',
 '/threads/edition-hotel-klcc-kuala-lumpur-persiaran-stonor-pro.1604589',
 '/threa

In [34]:
def get_href(url, forum):
    forum = forum.split('com/')[1]
    while True:
        try:
            r = requests.get(url, timeout = 5.0)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    a = [a_ for a_ in a if forum in a_ and '/page-' in a_]
    max_page = []
    for a_ in a:
        try:
            max_page.append(int(a_.split('/page-')[-1]))
        except:
            pass

    if len(max_page):
        max_page = max(max_page)
    else:
        max_page = -1
    
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    a = [a_ for a_ in a if '/threads/' in a_]
    a = list(set(['/'.join(a_.split('/')[:3]) for a_ in a]))
        
    return a, max_page

filtered, max_page = get_href(forums[0], forums[0])
filtered, max_page

(['/threads/bandar-cassia-batu-kawan-penang-u-c.1854698',
  '/threads/tun-razak-exchange-kuala-lumpur-jln-tun-razak.1135763',
  '/threads/kuching-isthmus-development-general-news-updates.973986',
  '/threads/vanke-kl-serani-row-redevelopment-kuala-lumpur-jalan-raja-chulan-pro.2073073',
  '/threads/seri-tanjung-pinang-gurney-bay-penang.552100',
  '/threads/ncer-northern-corridor-economic-region-penang-kedah-perlis-sime-darby.495680',
  '/threads/armani-hartajaya-bb-kuala-lumpur-jalan-raja-chulan-prep.2071347',
  '/threads/penang-international-commercial-city-penang-bayan-baru.1973689',
  '/threads/klang-valley-shopping-malls-proposals-news-updates.1498603',
  '/threads/merdeka-118-kuala-lumpur-changkat-stadium-city-centre.956606',
  '/threads/tropicana-windcity-genting-highlands.2230624',
  '/threads/kwasa-damansara-city-centre-selangor-sg-buloh.1522715',
  '/threads/sarawak-corridor-of-renewable-energy-score.581188',
  '/threads/edition-hotel-klcc-kuala-lumpur-persiaran-stonor-pro.1604

In [35]:
!mkdir forums

In [36]:
forums[0]

'https://www.skyscrapercity.com/forums/countdown-to-2030.332/'

In [ ]:
'https://www.skyscrapercity.com/forums/countdown-to-2030.332/page-2'

In [ ]:
max_worker = 30

forums = sorted(forums)
for i in tqdm(range(len(forums))):
    filename = os.path.join('forums', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    forum = forums[i]
    url = forum
    filtered, max_page = get_href(url, forum)
    for i in tqdm(range(2, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [(f'{url}page-{a}', forum) for a in aranged]
        
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(get_href, url[0], url[1]): url for url in urls}

            for future in as_completed(futures):
                filtered.extend(future.result()[0])
                
    with open(filename, 'w') as fopen:
        json.dump(filtered, fopen)

  0%|                                                                                                       | 0/1 [00:00<?, ?it/s]